In [1]:
!pip install azure-storage-blob


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.1/207.1 kB 11.0 MB/s eta 0:00:00


In [6]:
# -------------------------
# 1. Conexión a Azure Blob Storage
# -------------------------

from azure.storage.blob import BlobServiceClient
import os
# Imports para conexión a Azure Blob y manejo de archivos
from azure.storage.blob import BlobServiceClient
import os

# Imports de PySpark
from pyspark.sql.functions import col, to_timestamp, window, avg, count


from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("UberAnalyticsBlob") \
    .getOrCreate()


# Credenciales
conn_str = "DefaultEndpointsProtocol=https;AccountName=iesstsabbadbab;AccountKey=/Z4VcADF8fi/0zqf5v4aJk47k5MAUZFTVx7bkbdId3N0zG+UQv7bmA9Qr6ygGEGMEYwikrOBfRjk+AStl5SehA==;EndpointSuffix=core.windows.net"
container_name = "group4"
folder_prefix_rides = "ride_stream/part"
folder_prefix_traffic = "traffic_stream/part"

# Cliente
blob_service_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_service_client.get_container_client(container_name)

# Descarga de archivos Parquet
def descargar_parquet(folder_prefix):
    parquet_files = []
    for blob in container_client.list_blobs(name_starts_with=folder_prefix):
        if blob.name.endswith(".parquet"):
            print(f"⬇️  Descargando: {blob.name}")
            local_path = os.path.basename(blob.name)
            blob_client = container_client.get_blob_client(blob)
            with open(local_path, "wb") as f:
                f.write(blob_client.download_blob().readall())
            parquet_files.append(local_path)
    return parquet_files

# Descarga rides y traffic
parquet_rides = descargar_parquet(folder_prefix_rides)
parquet_traffic = descargar_parquet(folder_prefix_traffic)



# -------------------------
# 2. Carga de datos en Spark
# -------------------------

# Leer archivos Parquet descargados desde el blob
df_rides = spark.read.parquet(*parquet_rides)
df_traffic = spark.read.parquet(*parquet_traffic)

df_rides.printSchema()
df_traffic.printSchema()

df_rides.show(5)
df_traffic.show(5)



# -------------------------
# 3. Transformaciones con Spark SQL / PySpark API
# -------------------------
from pyspark.sql.functions import col, to_timestamp, window, avg, count

# Convertir a timestamp si aún no lo están
df_rides = df_rides.withColumn("timestamp_event", to_timestamp(col("timestamp_event")))
df_traffic = df_traffic.withColumn("timestamp", to_timestamp(col("timestamp")))


# -------------------------
# 4. Análisis e Insights
# -------------------------

# 4.1 Total de viajes cada 10 minutos
print("📊 Total de viajes cada 10 minutos:\n")

df_rides.groupBy(
    window(col("timestamp_event"), "10 minutes")
).agg(
    count("*").alias("num_rides")
).orderBy("window").show(truncate=False)


#4.2 Total de viajes por tipo de evento
print("📊 Total de viajes por tipo de evento cada 10 minutos:\n")

df_rides.groupBy(
    window(col("timestamp_event"), "10 minutes"),
    col("event_type")
).agg(
    count("*").alias("num_events")
).orderBy("window", "event_type").show(truncate=False)


#4.3 Precio promedio por tipo de Uber
print("💰 Precio promedio por tipo de Uber:\n")

df_rides.groupBy("uber_type").agg(
    avg("price").alias("avg_price")
).orderBy("avg_price", ascending=False).show(truncate=False)


# 4.4 Número de alertas por zona (desde traffic_df)
print("🚨 Número de alertas de tráfico por zona:\n")

df_traffic.groupBy("zone_id").agg(
    count("*").alias("num_alerts")
).orderBy("num_alerts", ascending=False).show(truncate=False)


#4.5 Promedio del multiplicador por nivel de tráfico
print("📈 Promedio de surge multiplier por nivel de tráfico:\n")

df_traffic.groupBy("traffic_level").agg(
    avg("surge_multiplier").alias("avg_surge")
).orderBy("traffic_level").show(truncate=False)


# -----------------------------------------
# EXTENSIONES PARA SUBIR A NIVEL INTERMEDIO / AVANZADO
# -----------------------------------------

from pyspark.sql.functions import unix_timestamp, stddev, mean, when, lit, abs, count, avg

# 1. Analizar viajes activos, completados, cancelados
print("📊 Viajes por tipo de evento:")
df_rides.groupBy("event_type").count().orderBy("event_type").show()

# 2. Calcular duración del viaje y tiempo de respuesta del conductor
if all(col_name in df_rides.columns for col_name in ["pickup_time", "dropoff_time", "request_time"]):
    df_rides = df_rides \
        .withColumn("driver_response", unix_timestamp("pickup_time") - unix_timestamp("request_time")) \
        .withColumn("ride_duration", unix_timestamp("dropoff_time") - unix_timestamp("pickup_time"))

    print("⏱️ Duración del viaje y tiempo de respuesta del conductor:")
    df_rides.select("ride_id", "driver_response", "ride_duration").show(5)

# 3. Calcular tasa de cancelación
total_rides = df_rides.count()
cancelled_rides = df_rides.filter(col("event_type") == "cancelled").count()
cancellation_rate = (cancelled_rides / total_rides) * 100
print(f"❌ Cancelaciones: {cancelled_rides} de {total_rides} viajes ({cancellation_rate:.2f}%)")

# 4. Detección de anomalías en precios (z-score)
if "price" in df_rides.columns:
    stats_df = df_rides.select(
        mean(col("price")).alias("mean_price"),
        stddev(col("price")).alias("stddev_price")
    ).collect()[0]

    mean_price = stats_df["mean_price"]
    stddev_price = stats_df["stddev_price"]

    df_rides = df_rides.withColumn(
        "is_price_anomaly",
        when(abs(col("price") - mean_price) > 3 * stddev_price, lit(True)).otherwise(lit(False))
    )

    print("⚠️ Anomalías detectadas en precios (z-score > 3):")
    df_rides.filter("is_price_anomaly").select("ride_id", "price").show(5)

# 5. Predicción simple de zonas con surge pricing alto
if "surge_multiplier" in df_traffic.columns:
    print("🚀 Zonas con surge pricing alto (histórico):")
    df_traffic.groupBy("zone_id").agg(avg("surge_multiplier").alias("avg_surge")) \
        .orderBy("avg_surge", ascending=False).show(5)

# 6. Identificación de posibles fraudes por ubicación frecuente
if "start_location" in df_rides.columns:
    print("🕵️ Ubicaciones con eventos anómalamente frecuentes:")
    df_rides.groupBy("start_location").agg(count("*").alias("event_count")) \
        .orderBy("event_count", ascending=False).show(5)




# 1. Eventos por tipo de Uber
print("\n📊 1. Número de eventos por tipo de Uber:")
df_rides.groupBy("uber_type").count().orderBy("count", ascending=False).show()

# 2. Filtrar solo viajes caros (mayores a 30€)
print("\n💸 2. Viajes caros (precio > 30€):")
df_rides.filter(col("price") > 30).select("ride_id", "uber_type", "price").show()

# 3. Precio promedio por tipo de Uber
print("\n💰 3. Precio promedio por tipo de Uber:")
df_rides.groupBy("uber_type").agg(avg("price").alias("avg_price")).orderBy("avg_price", ascending=False).show()

# 4. Viajes por franja horaria (mañana, tarde, noche, madrugada)
print("\n🕒 4. Distribución de viajes por franja horaria:")
from pyspark.sql.functions import hour, when
df_rides = df_rides.withColumn("hour", hour("timestamp_event"))
df_rides = df_rides.withColumn("time_slot", when((col("hour") >= 6) & (col("hour") < 12), "Mañana")
                                .when((col("hour") >= 12) & (col("hour") < 18), "Tarde")
                                .when((col("hour") >= 18) & (col("hour") < 24), "Noche")
                                .otherwise("Madrugada"))
df_rides.groupBy("time_slot").count().orderBy("time_slot").show()

# 5. Rutas más frecuentes
print("\n🗺️ 5. Rutas más frecuentes:")
df_rides.groupBy("start_location", "end_location").count().orderBy("count", ascending=False).show(10)

# 6. Viajes por día de la semana
print("\n📆 6. Distribución de viajes por día de la semana:")
from pyspark.sql.functions import date_format
df_rides = df_rides.withColumn("day_of_week", date_format("timestamp_event", "EEEE"))
df_rides.groupBy("day_of_week").count().orderBy("day_of_week").show()

# 7. Completados vs Cancelados
print("\n✅❌ 7. Viajes completados vs cancelados:")
df_rides.filter(col("event_type").isin("completed", "cancelled")) \
    .groupBy("event_type").count().orderBy("event_type").show()

# 8. Precio promedio por zona de inicio
print("\n📍 8. Precio promedio por zona de inicio:")
df_rides.groupBy("start_location").agg(avg("price").alias("avg_price")).orderBy("avg_price", ascending=False).show(10)

# 9. Evolución semanal del precio (df_rides)
print("\n📈 9. Evolución semanal del precio promedio:")
from pyspark.sql.functions import weekofyear
df_rides = df_rides.withColumn("week", weekofyear("timestamp_event"))
df_rides.groupBy("week").agg(avg("price").alias("avg_price")).orderBy("week").show()

# 10. Alertas por tipo de evento
print("\n🚨 10. Número de alertas por tipo de evento:")
df_traffic.groupBy("event_type").count().orderBy("count", ascending=False).show()

# 11. Alertas por zona
print("\n📍 11. Zonas con más alertas de tráfico:")
df_traffic.groupBy("zone_id").count().orderBy("count", ascending=False).show(10)

# 12. Nivel promedio de congestión por zona
print("\n📊 12. Nivel promedio de congestión (surge multiplier) por zona:")
df_traffic.groupBy("zone_id").agg(avg("surge_multiplier").alias("avg_surge")) \
    .orderBy("avg_surge", ascending=False).show(10)

# 13. ¿Qué tipo de Uber se usa más en zonas con tráfico severo?
print("\n🚦 13. Tipo de Uber más utilizado en zonas con tráfico severo:")
severe_zones = df_traffic.filter(col("traffic_level") == "severe").select("zone_id").distinct()
df_rides_severe = df_rides.join(severe_zones, df_rides["start_location"] == severe_zones["zone_id"])
df_rides_severe.groupBy("uber_type").count().orderBy("count", ascending=False).show()



⬇️  Descargando: ride_stream/part-00000-49a9a2de-9db2-490b-b6d4-ab157299a5e1-c000.snappy.parquet
⬇️  Descargando: ride_stream/part-00000-5ee4e9f2-a419-4935-92dc-e5fca90de581-c000.snappy.parquet
⬇️  Descargando: ride_stream/part-00000-8c572137-38fd-410c-9517-9ad24b5faf1c-c000.snappy.parquet
⬇️  Descargando: ride_stream/part-00000-b5e3df0f-e2d5-4162-9dc0-9eedf8f0bd86-c000.snappy.parquet
⬇️  Descargando: ride_stream/part-00000-e611da15-f07f-4d3c-bfad-8552a164d778-c000.snappy.parquet
⬇️  Descargando: traffic_stream/part-00000-9c627dd1-f19d-4eed-97b5-7e8c090e4d77-c000.snappy.parquet
⬇️  Descargando: traffic_stream/part-00000-9ca600c0-041d-4dc1-9933-ee0353739c8d-c000.snappy.parquet
⬇️  Descargando: traffic_stream/part-00000-aafee65b-4951-4683-85d5-f6608c4b8218-c000.snappy.parquet
⬇️  Descargando: traffic_stream/part-00000-ca4b202e-fce1-472c-be7d-58c44fbbc5d8-c000.snappy.parquet
⬇️  Descargando: traffic_stream/part-00000-fd8e2b10-f331-44a7-90c4-a6bc6f5740da-c000.snappy.parquet
root
 |-- event